In [1]:
import textwrap # for print long text in wrapped lines
from pprint import pprint # for print pretty json

def format_text(text, width=80, indent=0):
    # Split text into lines
    lines = text.split('\n')
    # Format each line, keeping short lines unchanged
    formatted_lines = []
    for line in lines:
        if len(line) <= width - indent:
            # Keep short lines as is, just add indent
            formatted_lines.append(" "*indent + line)
        else:
            # Wrap long lines
            wrapped = textwrap.fill(line, width=width, initial_indent=" "*indent, subsequent_indent=" "*indent)
            formatted_lines.append(wrapped)
    return '\n'.join(formatted_lines)

## Get embeddings and vector store, do semantic search

In [2]:
import getpass
import os

# for convert text to embedding vectors, so that we can use it for similarity search and RAG
from langchain_openai import OpenAIEmbeddings 

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# from langchain_ollama import OllamaEmbeddings # alternatively use open source models on your own server
# embeddings = OllamaEmbeddings(model="qwen2.5")

from langchain_unstructured import UnstructuredLoader # for load data from unstructured files 
from langchain_text_splitters import RecursiveCharacterTextSplitter # for split text into chunks

In [3]:
# to use unstructured, we need to set it up following: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file/

file_paths = [
  "../hardware/datasheets/TB6612FNG.pdf"
]
loader = UnstructuredLoader(
  file_paths,
  chunking_strategy="basic",
  max_characters=1000000,
  include_orig_elements=False,
  )


docs = loader.load()
docs[0]

pprint(docs[0].metadata)

print("Number of LangChain documents:", len(docs))
print("Length of text in the document:", len(docs[0].page_content))

/global/homes/s/sulinl/.conda/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: pikepdf C++ to Python logger bridge initialized


{'category': 'CompositeElement',
 'element_id': '7618d194468c1f60378ca5d56cd48d8a',
 'file_directory': '../hardware/datasheets',
 'filename': 'TB6612FNG.pdf',
 'filetype': 'application/pdf',
 'languages': ['eng'],
 'last_modified': '2024-12-03T13:04:22',
 'page_number': 1,
 'source': '../hardware/datasheets/TB6612FNG.pdf'}
Number of LangChain documents: 1
Length of text in the document: 14196


In [4]:
print(format_text(docs[0].page_content[:200]))


TB6612FNG

Toshiba Bi-CD Integrated Circuit Silicon Monolithic

TB6612FNG

Driver IC for Dual DC motor

TB6612FNG is a driver IC for DC motor with output transistor in LD MOS structure
with low ON-res


In [5]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)


19

In [6]:
print(format_text(all_splits[2].page_content))


Pin Functions

No.

Pin Name

1

AO1

2

AO1

3

PGND1

4

PGND1

5

AO2

6

AO2

7

BO2

8

BO2

9

PGND2

10

PGND2

11

BO1

12

BO1

13

VM2

14

VM3

15

PWMB

16

BIN2

17

BIN1

18

GND

19

STBY

20

Vcc

21

AIN1

22

AIN2

23

PWMA

24

VM1

PWMA

AIN2

AIN1

Vcc

STBY

GND

BIN1 BIN2 PWMB

VM3 VM2

23

22

21

20

19

18

17

16

15

14

13

STB

Control Logic A

UVLO

Control Logic B

TSD

H-SW A

H-SW B

2

3

4

5

6

7

8

9

10

11

12

AO1

PGND1

PGND1

AO2

AO2

BO2

BO2 PGND2

PGND2

BO1

BO1

I/O

Function

O

ch A output1

⎯

Power GND 1

O

ch A output2

O

ch B output2

⎯

Power GND 2

O

ch B output1

⎯

Motor supply (2.5 V to 13.5 V)

I

ch B PWM input / 200 kΩ pull-down at internal

I

ch B input 2 / 200 kΩ pull-down at internal

I

ch B input 1 / 200 kΩ pull-down at internal

⎯

Small signal GND

I

“L”=standby / 200 kΩ pull-down at internal

⎯

Small signal supply

I

ch A input 1 / 200 kΩ pull-down at internal

I


In [7]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Generated vectors of length 3072

[-0.01971038058400154, -0.0030921949073672295, -0.009017327800393105, -0.017691269516944885, 0.014422230422496796, 0.031152017414569855, -0.013872811570763588, 0.018130803480744362, -0.03315739333629608, 0.031179487705230713]


In [8]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
ids = vector_store.add_documents(documents=all_splits)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [9]:
# an example of similarity search fo a question
results = vector_store.similarity_search(
    "What is the function of pin AO1?"
)

print("Number of retrieved results:", len(results))
# print the first result
print(format_text(results[0].page_content))

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Number of retrieved results: 4
Pin Functions

No.

Pin Name

1

AO1

2

AO1

3

PGND1

4

PGND1

5

AO2

6

AO2

7

BO2

8

BO2

9

PGND2

10

PGND2

11

BO1

12

BO1

13

VM2

14

VM3

15

PWMB

16

BIN2

17

BIN1

18

GND

19

STBY

20

Vcc

21

AIN1

22

AIN2

23

PWMA

24

VM1

PWMA

AIN2

AIN1

Vcc

STBY

GND

BIN1 BIN2 PWMB

VM3 VM2

23

22

21

20

19

18

17

16

15

14

13

STB

Control Logic A

UVLO

Control Logic B

TSD

H-SW A

H-SW B

2

3

4

5

6

7

8

9

10

11

12

AO1

PGND1

PGND1

AO2

AO2

BO2

BO2 PGND2

PGND2

BO1

BO1

I/O

Function

O

ch A output1

⎯

Power GND 1

O

ch A output2

O

ch B output2

⎯

Power GND 2

O

ch B output1

⎯

Motor supply (2.5 V to 13.5 V)

I

ch B PWM input / 200 kΩ pull-down at internal

I

ch B input 2 / 200 kΩ pull-down at internal

I

ch B input 1 / 200 kΩ pull-down at internal

⎯

Small signal GND

I

“L”=standby / 200 kΩ pull-down at internal

⎯

Small signal supply

I

ch A input 1 / 200 kΩ pull-down at internal

I


In [10]:
print(format_text(results[1].page_content))

Small faced package(SSOP24: 0.65 mm Lead pitch)

Response to Pb free packaging

This product has a MOS structure and is sensitive to electrostatic discharge.
When handling this product,

ensure that the environment is protected against electrostatic discharge by
using an earth strap, a conductive mat and an ionizer. Ensure also that the
ambient temperature and relative humidity are maintained at reasonable levels.

The TB6612FNG is a Pb-free product. The following conditions apply to
solderability: *Solderability 1. Use of Sn-37Pb solder bath

solder bath temperature = 230°C *dipping time = 5 seconds *number of times =
once *use of R-type flux

2. Use of Sn-3.0Ag-0.5Cu solder bath *solder bath temperature = 245°C *dipping
time = 5 seconds

1

2008-05-09

Block Diagram

VM1

24

1

AO1

Pin Functions

No.

Pin Name

1

AO1

2

AO1

3

PGND1

4

PGND1

5

AO2

6

AO2

7

BO2

8

BO2

9

PGND2

10

PGND2

11

BO1

12

BO1

13

VM2

14

VM3

15

PWMB

16

BIN2

17

BIN1

18

GND

19


In [11]:
import getpass
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatOpenAI(model="o1-preview") # much stronger reasoning

## Build the RAG with LangGraph

In [12]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt") # details in https://smith.langchain.com/hub/rlm/rag-prompt


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/global/homes/s/sulinl/.conda/envs/langchain/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
response = graph.invoke({"question": f"Output the pin mapping of TB6612FNG in json format, including the pin name, number, and function."})

# print(f'Context: {response["context"]}\n\n')
print(format_text(response["answer"]))

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```json
{
  "pins": [
    {"name": "AO1", "number": 1, "function": "ch A output1"},
    {"name": "AO1", "number": 2, "function": "ch A output1"},
    {"name": "PGND1", "number": 3, "function": "Power GND 1"},
    {"name": "PGND1", "number": 4, "function": "Power GND 1"},
    {"name": "AO2", "number": 5, "function": "ch A output2"},
    {"name": "AO2", "number": 6, "function": "ch A output2"},
    {"name": "BO2", "number": 7, "function": "ch B output2"},
    {"name": "BO2", "number": 8, "function": "ch B output2"},
    {"name": "PGND2", "number": 9, "function": "Power GND 2"},
    {"name": "PGND2", "number": 10, "function": "Power GND 2"},
    {"name": "BO1", "number": 11, "function": "ch B output1"},
    {"name": "BO1", "number": 12, "function": "ch B output1"},
    {"name": "VM2", "number": 13, "function": "Motor supply (2.5 V to 13.5 V)"},
    {"name": "VM3", "number": 14, "function": "Motor supply (2.5 V to 13.5 V)"},
    {"name": "PWMB", "number": 15, "function": "ch B PWM input / 

In [14]:
# an example of using the RAG for question answering

response = graph.invoke({"question": "What does this datasheet say about how to control the motor?"})

print(f'Context: {response["context"]}\n\n')
print(format_text(response["answer"]))

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Context: [Document(id='8570a22c-ed76-4ada-9c04-3994f174fb78', metadata={'source': '../hardware/datasheets/TB6612FNG.pdf', 'file_directory': '../hardware/datasheets', 'filename': 'TB6612FNG.pdf', 'languages': ['eng'], 'last_modified': '2024-12-03T13:04:22', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'CompositeElement', 'element_id': '7618d194468c1f60378ca5d56cd48d8a', 'start_index': 2998}, page_content='76.2 mm × 114.3 mm t=1.6 mm Cu ≥ 30% in PCB monting\n\nOperating temperature\n\nTopr\n\n20 to 85\n\n℃\n\nStorage temperature\n\nTstg\n\n55 to 150\n\n℃\n\nOperating Range (Ta=-20～85℃)\n\nCharacteristics\n\nSymbol\n\nMin\n\nTyp.\n\nMax\n\nUnit\n\nRemarks\n\nSupply voltage\n\nOutput current (H-SW)\n\nVCC VM\n\nIOUT\n\n2.7\n\n2.5 ⎯\n\n⎯\n\n3\n\n5 ⎯\n\n⎯\n\n5.5\n\n13.5\n\n1.0\n\n0.4\n\nV\n\nV\n\nA\n\nVM ≥ 4.5 V 4.5 V > VM ≥ 2.5 V Without PWM Operation\n\nSwitching frequency\n\nfPWM\n\n⎯\n\n⎯\n\n100\n\nkHz\n\nInput pin; 入 力 端 子 ; IN1,IN2,PWM,STBY\n\nOutput pin; 出 力 端 子 ; O1,O

In [15]:
print(response["answer"])
summary = response["answer"]
# print(textwrap.fill(response["answer"], width=80))

The datasheet specifies that the motor can be controlled using two input signals, IN1 and IN2, which allow for four modes: clockwise (CW), counterclockwise (CCW), short brake, and stop. Additionally, a PWM input can be utilized for speed control. The standby mode can be activated by applying a low signal to the STBY pin.


In [16]:
response = graph.invoke({"question": f"Give this code {summary}, for Arduino Nano, output the C code for the PWM driver, make sure the PWM driver has frequency of 5000 Hz."})

# print(f'Context: {response["context"]}\n\n')
print(format_text(response["answer"]))


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Here is a simple C code for the PWM driver for Arduino Nano using the TB6612FNG
motor driver with a PWM frequency of 5000 Hz:

```c
#define IN1 3
#define IN2 4
#define PWM 5
#define STBY 6

void setup() {
  pinMode(IN1, OUTPUT);
  pinMode(IN2, OUTPUT);
  pinMode(PWM, OUTPUT);
  pinMode(STBY, OUTPUT);
  digitalWrite(STBY, LOW);  // Activate standby mode
}

void loop() {
  digitalWrite(IN1, HIGH);   // Set direction CW
  digitalWrite(IN2, LOW);
  analogWrite(PWM, 128);     // Set speed (0-255)
  delay(1000);                // Run for 1 second
  // Add more functionality as needed
}
```
This code sets up the motor driver and controls the motor in the clockwise
direction with a PWM speed of approximately 50%. Adjust `analogWrite(PWM,
value)` to change the speed as required.


In [17]:
sample_code = response["answer"]
response = graph.invoke({"question": f"Give original code {sample_code}, for Arduino Nano, justify values you used to make sure the PWM driver has frequency of 5000 Hz."})

# print(f'Context: {response["context"]}\n\n')
print(format_text(response["answer"]))

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


To achieve a PWM frequency of 5000 Hz using the Arduino Nano and the TB6612FNG
motor driver, you can set the PWM pin to use a fast PWM mode. The default PWM
frequency for pins 5 and 6 on the Arduino Nano is approximately 976 Hz, but by
manipulating the prescaler settings of the timer, you can increase the frequency
to 5000 Hz. This requires configuring the timer registers directly, which is not
shown in the original code snippet provided.


In [18]:
for step in graph.stream(
    {"question": "Output the pin mapping of TB6612FNG in json format, including the pin name, number, and function."}, stream_mode="updates"
):
    print(f"{step}\n\n----------------\n")

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


{'retrieve': {'context': [Document(id='a01cb1e8-85ac-4766-ae37-40d1c34ce970', metadata={'source': '../hardware/datasheets/TB6612FNG.pdf', 'file_directory': '../hardware/datasheets', 'filename': 'TB6612FNG.pdf', 'languages': ['eng'], 'last_modified': '2024-12-03T13:04:22', 'page_number': 1, 'filetype': 'application/pdf', 'category': 'CompositeElement', 'element_id': '7618d194468c1f60378ca5d56cd48d8a', 'start_index': 632}, page_content='Small faced package(SSOP24: 0.65 mm Lead pitch)\n\nResponse to Pb free packaging\n\nThis product has a MOS structure and is sensitive to electrostatic discharge. When handling this product,\n\nensure that the environment is protected against electrostatic discharge by using an earth strap, a conductive mat and an ionizer. Ensure also that the ambient temperature and relative humidity are maintained at reasonable levels.\n\nThe TB6612FNG is a Pb-free product. The following conditions apply to solderability: *Solderability 1. Use of Sn-37Pb solder bath\n\ns

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'generate': {'answer': '```json\n{\n  "pins": [\n    {"pin_name": "AO1", "number": 1, "function": "ch A output1"},\n    {"pin_name": "AO1", "number": 2, "function": "ch A output1"},\n    {"pin_name": "PGND1", "number": 3, "function": "Power GND 1"},\n    {"pin_name": "PGND1", "number": 4, "function": "Power GND 1"},\n    {"pin_name": "AO2", "number": 5, "function": "ch A output2"},\n    {"pin_name": "AO2", "number": 6, "function": "ch A output2"},\n    {"pin_name": "BO2", "number": 7, "function": "ch B output2"},\n    {"pin_name": "BO2", "number": 8, "function": "ch B output2"},\n    {"pin_name": "PGND2", "number": 9, "function": "Power GND 2"},\n    {"pin_name": "PGND2", "number": 10, "function": "Power GND 2"},\n    {"pin_name": "BO1", "number": 11, "function": "ch B output1"},\n    {"pin_name": "BO1", "number": 12, "function": "ch B output1"},\n    {"pin_name": "VM2", "number": 13, "function": "Motor supply"},\n    {"pin_name": "VM3", "number": 14, "function": "Motor supply"},\n   